In [23]:
import numpy as np
import pandas as pd
from typing import Tuple
import matplotlib.pyplot as plt
import os

####
from catboost import CatBoostClassifier

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.feature_extraction.text import TfidfVectorizer

from collections import Counter

from nltk.corpus import stopwords

stop_words = set(stopwords.words("russian"))

In [8]:
import multiprocessing as mp
from typing import Union
import mlflow
import os
import re
import sklearn
import string

sklearn.set_config(transform_output="pandas")

In [9]:
# if __name__ == "__main__":
#     mp.set_start_method("spawn", force=True)
# DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
# # GENERATOR = (
# #     torch.Generator(device=DEVICE) if torch.cuda.is_available() else torch.Generator()
# # )
# GENERATOR = torch.Generator()

# use_mlflow = True
# mlflow.set_tracking_uri("http://localhost:5000")
CURR_DIR = os.curdir

In [10]:
stop_words.difference_update({"не", "нет", "без"})

In [11]:
def data_preprocessing(text: str) -> str:
    """preprocessing string: lowercase, removing html-tags, punctuation,
                            stopwords, digits

    Args:
        text (str): input string for preprocessing

    Returns:
        str: preprocessed string
    """

    text = text.lower()
    text = re.sub("<.*?>", "", text)  # html tags
    text = "".join(
        [c for c in text if c not in string.punctuation]
    )  # Remove punctuation
    text = " ".join([word for word in text.split() if word not in stop_words])
    text = " ".join([word for word in text.split() if not word.isdigit()])
    return text

In [19]:
df = pd.read_json(
    os.path.join(CURR_DIR, "..", "data", "healthcare_facilities_reviews.jsonl"),
    lines=True,
)
labels = df["sentiment"].copy().apply(lambda x: 1 if x == "positive" else 0)
df["labels"] = labels
data = df.loc[:, ["content", "labels"]].copy()
print(data.shape)
data.head()

(70597, 2)


,content,labels
0,Огромное спасибо за чудесное удаление двух зуб...,1
1,Хочу выразить особую благодарность замечательн...,1
2,Добрый вечер! Хотелось бы поблагодарить сотруд...,1
3,Женщины советского образца в регистратуре не и...,0
4,У меня с детства очень плохие зубы (тонкая и х...,1


In [20]:
data["content_processed"] = data["content"].copy().apply(data_preprocessing)
data.head()

,content,labels,content_processed
0,Огромное спасибо за чудесное удаление двух зуб...,1,огромное спасибо чудесное удаление двух зубов ...
1,Хочу выразить особую благодарность замечательн...,1,хочу выразить особую благодарность замечательн...
2,Добрый вечер! Хотелось бы поблагодарить сотруд...,1,добрый вечер хотелось поблагодарить сотруднико...
3,Женщины советского образца в регистратуре не и...,0,женщины советского образца регистратуре не име...
4,У меня с детства очень плохие зубы (тонкая и х...,1,детства очень плохие зубы тонкая хрупкая эмаль...


In [21]:
data["labels"].value_counts()

labels
1    41419
0    29178
Name: count, dtype: int64

In [28]:
X_train, X_valid, y_train, y_valid = train_test_split(
    data["content_processed"], data["labels"], test_size=0.2, random_state=42
)

In [29]:
vectorizer = TfidfVectorizer(
    max_features=5000,  # ограничим размер словаря (опционально)
    ngram_range=(1, 2),  # используем униграммы и биграммы
    stop_words=list(stop_words),  # можно указать 'russian' для русских стоп-слов
)
X_train = vectorizer.fit_transform(X_train)
X_valid = vectorizer.transform(X_valid)

In [30]:
model = CatBoostClassifier(
    iterations=300,  # число деревьев
    depth=6,  # глубина
    learning_rate=0.1,
    eval_metric="Accuracy",
    verbose=50,
    random_seed=42,
)

In [31]:
model.fit(X_train, y_train, eval_set=(X_valid, y_valid), use_best_model=True)

0:	learn: 0.7969793	test: 0.7934136	best: 0.7934136 (0)	total: 314ms	remaining: 1m 33s
50:	learn: 0.8648653	test: 0.8616856	best: 0.8616856 (50)	total: 13.3s	remaining: 1m 4s
100:	learn: 0.8922747	test: 0.8861898	best: 0.8861898 (100)	total: 25.5s	remaining: 50.2s
150:	learn: 0.9051118	test: 0.8955382	best: 0.8955382 (150)	total: 37.9s	remaining: 37.4s
200:	learn: 0.9154877	test: 0.9026912	best: 0.9026912 (200)	total: 50.1s	remaining: 24.7s
250:	learn: 0.9233139	test: 0.9072946	best: 0.9072946 (248)	total: 1m 2s	remaining: 12.1s
299:	learn: 0.9300246	test: 0.9109065	best: 0.9109065 (298)	total: 1m 13s	remaining: 0us

bestTest = 0.9109065156
bestIteration = 298

Shrink model to first 299 iterations.


In [32]:
# 5. Оценка качества
y_pred = model.predict(X_valid)
print(classification_report(y_valid, y_pred))

              precision    recall  f1-score   support

           0       0.88      0.91      0.89      5778
           1       0.93      0.91      0.92      8342

    accuracy                           0.91     14120
   macro avg       0.91      0.91      0.91     14120
weighted avg       0.91      0.91      0.91     14120



In [ ]:
# os.mkdir(os.path.join(CURR_DIR, "weights", "Catboost+TFIDF"))

In [34]:
import joblib

model.save_model(
    os.path.join(CURR_DIR, "weights", "Catboost+TFIDF", "catboost_model.cbm")
)
joblib.dump(
    vectorizer,
    os.path.join(CURR_DIR, "weights", "Catboost+TFIDF", "tfidf_vectorizer.pkl"),
)

['./weights/Catboost+TFIDF/tfidf_vectorizer.pkl']